In [1]:
import pandas as pd
import pickle
from sklearn.model_selection import LeaveOneOut
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn import svm
from numpy import mean
from numpy import absolute
from numpy import sqrt
from numpy import std

In [2]:
# loading the data from R (matrix with all the variables)


df = pd.read_csv('taxonomic_matrix2.csv')
df = df.drop('Unnamed: 0', 1)

df


,X.Deinococcus.soli..Cha.et.al..2014,Abaca.bunchy.top.virus,Abalone.herpesvirus.Victoria.AUS.2009,Abalone.shriveling.syndrome.associated.virus,Abutilon.mosaic.virus,Acanthamoeba.polyphaga.mimivirus,Acanthocystis.turfacea.Chlorella.virus.1,Acaryochloris.marina,Acetoanaerobium.sticklandii,Acetobacter.pasteurianus,...,haloarchaeon.HTSR1,halophilic.archaeon.DL31,methanogenic.archaeon.ISO4.H5,secondary.endosymbiont.of.Ctenarytaina.eucalypti,secondary.endosymbiont.of.Heteropsylla.cubana,synthetic.construct,uncultured.Termite.group.1.bacterium,uncultured.phage.WW.nAnB,unidentified.adenovirus,condition2
0,0.045071,0.000000,0.000094,0.000000,0.000031,0.003222,0.000375,0.000156,0.033529,0.030589,...,0.005474,0.007726,0.016170,0.004692,0.004004,0.490337,0.000000,0.000000,0.000000,1
1,0.006422,0.000000,0.000019,0.000048,0.000000,0.000347,0.000019,0.000010,0.008088,0.004776,...,0.000992,0.002003,0.002696,0.000356,0.000337,0.269065,0.000010,0.000000,0.000000,1
2,0.006126,0.000000,0.000347,0.000101,0.000000,0.000470,0.000000,0.000011,0.006439,0.004863,...,0.001096,0.001375,0.003030,0.000872,0.000201,3.447051,0.000000,0.000011,0.000022,1
3,0.021681,0.000000,0.000047,0.000000,0.000000,0.000912,0.000094,0.000140,0.015086,0.009940,...,0.002900,0.005052,0.005450,0.003789,0.002432,0.574490,0.000023,0.000164,0.000000,1
4,0.016576,0.000000,0.000055,0.000027,0.000000,0.002294,0.000109,0.000055,0.043092,0.022638,...,0.002430,0.004014,0.004260,0.004096,0.002922,0.253666,0.000164,0.000000,0.000000,1
5,0.013644,0.000000,0.000169,0.000000,0.000000,0.001563,0.000507,0.000000,0.013939,0.017741,...,0.001563,0.012588,0.026822,0.001985,0.001098,0.664352,0.000042,0.000000,0.000000,1
6,0.011296,0.000000,0.000299,0.000026,0.000000,0.001040,0.000065,0.000091,0.007799,0.010256,...,0.002457,0.003549,0.003640,0.001014,0.000598,0.040804,0.000039,0.000000,0.000000,1
7,0.025251,0.000000,0.000119,0.000024,0.000000,0.002076,0.000740,0.000048,0.028306,0.016301,...,0.003413,0.006468,0.008855,0.004606,0.002434,0.576195,0.000000,0.000000,0.000000,1
8,0.048896,0.000000,0.000129,0.000000,0.000000,0.005171,0.000065,0.000000,0.023495,0.021879,...,0.008209,0.011796,0.012636,0.002133,0.002327,0.452764,0.000000,0.000485,0.003781,1
9,0.007688,0.000000,0.000022,0.000022,0.000000,0.000648,0.000223,0.000045,0.013521,0.013588,...,0.001698,0.002972,0.006168,0.001743,0.001184,2.277805,0.000022,0.000000,0.000000,1


In [3]:
## create the data

# Independent variable
X = df.drop(['condition2'], axis = 1) # removed last column (condition)

# Target variable
y = df.iloc[:,-1] # explicity selecting the last column


# the lower the value the better and 0 means the model is perfect


In [4]:
## LOOCV

cv = LeaveOneOut()
model = RandomForestClassifier()

# use the model to predict the response value of the one observation
# left out of the model and calculate the mean squared error (MSE)
scores = cross_val_score(model,  X , y , cv = cv, scoring='neg_mean_squared_error',n_jobs = -1) 
scores_acc = cross_val_score(model,  X , y , cv = cv, scoring='accuracy',n_jobs = -1) 


#view RMSE
print('mean squared error: ' + str(sqrt(mean(absolute(scores)))))
# accuracy
print('Accuracy: %.3f (%.3f)' % (mean(scores_acc), std(scores_acc)))

mean squared error: 0.7416198487095663
Accuracy: 0.550 (0.497)


In [5]:
## Store the model in Pickle format

# To fit our stimator (call fit in it explicity with provided dataset)
model.fit(X, y)
# To serialize
import pickle


with open('modelRF.pkl', 'wb') as fid:
    pickle.dump(model, fid, protocol = 2)
    
# To deserialize estimator later
with open('modelRF.pkl', 'rb') as fid:
    model = pickle.load(fid)




In [6]:
# Perform again LOOCV but with the features selected as important by Boruta

# Get the matrix with the features selected as important
data_FS = df[['Arthrobacter.phage.KellEzio','Escherichia.phage.PBECO.4','Flexistipes.sinusarabici',
             'Lactococcus.phage.50101','Salmonella.phage.vB_SosS_Oslo','Cellulophaga.phage.phi38.1',
             'Escherichia.phage.vB_EcoM_Alf5','Geobacillus.sp..JS12','Mycobacterium.phage.Charlie',
             'Yellowstone.lake.mimivirus','condition2']]

# Independent variable
X_FS = data_FS.drop(['condition2'], axis = 1) # removed last column (condition)

# Target variable
y_FS = data_FS.iloc[:,-1] # explicity selecting the last column

# Perform LOOCV
cv_FS = LeaveOneOut()
model_FS = RandomForestClassifier()

# use the model to predict the response value of the one observation
# left out of the model and calculate the mean squared error (MSE)
scores_FS = cross_val_score(model_FS,  X_FS , y_FS , cv = cv_FS, scoring='neg_mean_squared_error',n_jobs = -1) 
scores_FSA = cross_val_score(model_FS,  X_FS , y_FS , cv = cv_FS, scoring='accuracy',n_jobs = -1) 


#view RMSE
print('mean squared error: ' + str(sqrt(mean(absolute(scores_FS)))))
# accuracy
print('Accuracy: %.3f (%.3f)' % (mean(scores_FSA), std(scores_FSA)))

mean squared error: 0.3535533905932738
Accuracy: 0.850 (0.357)


In [7]:
# Save Random Forest model (with the features selected)

# To fit our stimator (call fit in it explicity with provided dataset)
model_FS.fit(X_FS, y_FS)
# To serialize
import pickle


with open('modelRF_FS.pkl', 'wb') as fid:
    pickle.dump(model_FS, fid, protocol = 2)
    
# To deserialize estimator later
with open('modelRF_FS.pkl', 'rb') as fid:
    model_FS = pickle.load(fid)

In [8]:
# SVM (all features)

# SVM cross validation (linear support vector classifier)
cv = LeaveOneOut()
modelSVM = SVC()

# use the model to predict the response value of the one observation
# left out of the model and calculate the mean squared error (MSE)
scores_svm = cross_val_score(modelSVM,  X , y , cv = cv, scoring='neg_mean_squared_error',n_jobs = -1) 
scores_svmAcc = cross_val_score(modelSVM,  X , y , cv = cv, scoring='accuracy',n_jobs = -1) 


#view RMSE
print('mean squared error (all features): ' + str(sqrt(mean(absolute(scores_svm)))))
# accuracy
print('Accuracy: %.3f (%.3f) (all features)' % (mean(scores_svmAcc), std(scores_svmAcc)))



mean squared error (all features): 0.689202437604511
Accuracy: 0.525 (0.499) (all features)


In [9]:
# SVM (after feature selection)

# SVM cross validation (linear support vector classifier)
cv_svmFS = LeaveOneOut()
modelSVM_FS = SVC()

# use the model to predict the response value of the one observation
# left out of the model and calculate the mean squared error (MSE)
scores_svm_FS = cross_val_score(modelSVM_FS,  X_FS , y_FS , cv = cv_svmFS, scoring='neg_mean_squared_error',n_jobs = -1) 
scores_svm_FSAcc = cross_val_score(modelSVM_FS,  X_FS , y_FS , cv = cv_svmFS, scoring='accuracy',n_jobs = -1) 


#view RMSE
print('mean squared error: ' + str(sqrt(mean(absolute(scores_svm_FS)))))
# accuracy
print('Accuracy: %.3f (%.3f)' % (mean(scores_svm_FSAcc), std(scores_svm_FSAcc)))



mean squared error: 0.6708203932499369
Accuracy: 0.550 (0.497)


In [ ]:
## The below code transforms our taxonomic matrix (with the features selected) and the metadata to the same
# format as GAIA works.



In [ ]:
## EXAMPLE METADATA INPUT:
# matrix: taxons as rows and samples as columns
# metadata: 

#Sample names   condition1
#SRR7423233_1   0
#SRR7423234_1   0
#SRR7423253_1   0
#SRR7423235_1   1
#SRR7423236_1   1
#SRR7423237_1   1


In [10]:
# transform our original metadata into GAIA format metadata

# initialize list of lists
data = [['SRR15853012', 1], ['SRR15852941', 1], ['SRR15852948', 1], ['SRR15853009', 1 ], ['SRR15853002',1],
       ['SRR15852986',1 ], ['SRR15852987', 1],['SRR15852998',1 ], ['SRR15853011',1 ], ['SRR15853000',1 ], ['SRR15853017',1 ],
       ['SRR15853005',1 ], ['SRR15852988', 1], ['SRR15852994',1 ], ['SRR15852995', 1], ['SRR15852997', 1], ['SRR15852989', 1], ['SRR15852990', 1],
       ['SRR15853001',1 ], ['SRR15852992', 1], ['SRR15852951',0 ], ['SRR15852922', 0], ['SRR15852955',0 ], ['SRR15852956',0 ], ['SRR15852962',0 ], ['SRR15852963',0 ], ['SRR15852982',0 ],
       ['SRR15853026', 0], ['SRR15853027',0 ], ['SRR15852932',0 ], ['SRR15852950', 0], ['SRR15852952',0 ], ['SRR15852954', 0], ['SRR15852957', 0], ['SRR15852960',0 ], ['SRR15852993', 0],
       ['SRR15852934',0 ], ['SRR15852968',0 ], ['SRR15852969', 0], ['SRR15853028',0 ]]
 
# Create the pandas DataFrame
dat = pd.DataFrame(data, columns = ['Sample names', 'condition1'])
 
# print dataframe.
dat

#pd.DataFrame(dat).to_csv("metadata.csv")

,Sample names,condition1
0,SRR15853012,1
1,SRR15852941,1
2,SRR15852948,1
3,SRR15853009,1
4,SRR15853002,1
5,SRR15852986,1
6,SRR15852987,1
7,SRR15852998,1
8,SRR15853011,1
9,SRR15853000,1


In [11]:
# Transform taxonomic matrix into Gaia input matrix format
# Matrix preprocessing

# remove last column (condition2)

dataG = data_FS.drop(['condition2'], axis = 1)

# transpose the matrix
dataG


,Arthrobacter.phage.KellEzio,Escherichia.phage.PBECO.4,Flexistipes.sinusarabici,Lactococcus.phage.50101,Salmonella.phage.vB_SosS_Oslo,Cellulophaga.phage.phi38.1,Escherichia.phage.vB_EcoM_Alf5,Geobacillus.sp..JS12,Mycobacterium.phage.Charlie,Yellowstone.lake.mimivirus
0,0.000156,0.000407,0.000000,0.000094,0.000031,0.000000,0.000000,0.020768,0.000000,0.000031
1,0.000048,0.000058,0.000106,0.000029,0.000019,0.000000,0.000000,0.001666,0.000000,0.000029
2,0.000011,0.000056,0.000034,0.000078,0.000268,0.000000,0.000000,0.001778,0.000000,0.000022
3,0.000023,0.000070,0.000000,0.000000,0.000023,0.000000,0.000023,0.004491,0.000000,0.000304
4,0.000055,0.000355,0.000055,0.000109,0.000055,0.000000,0.000000,0.008493,0.000027,0.000027
5,0.000127,0.000000,0.000127,0.000042,0.000000,0.000000,0.000042,0.045619,0.000000,0.000169
6,0.000078,0.000065,0.000117,0.000221,0.000078,0.000000,0.000000,0.002275,0.000000,0.000052
7,0.000072,0.000430,0.000095,0.000000,0.000000,0.000000,0.000024,0.007470,0.000000,0.000143
8,0.000129,0.000485,0.000000,0.000032,0.000032,0.000000,0.000032,0.031929,0.000032,0.000065
9,0.000179,0.000313,0.000022,0.000112,0.000000,0.000000,0.000000,0.005252,0.000000,0.000022


In [12]:
dataG = dataG.transpose()
dataG



,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
Arthrobacter.phage.KellEzio,0.000156,0.000048,0.000011,0.000023,0.000055,0.000127,0.000078,0.000072,0.000129,0.000179,...,0.000000,0.000000,0.000144,0.000020,0.000029,0.000000,0.000020,0.000034,0.000028,0.000210
Escherichia.phage.PBECO.4,0.000407,0.000058,0.000056,0.000070,0.000355,0.000000,0.000065,0.000430,0.000485,0.000313,...,0.000232,0.000079,0.000083,0.000061,0.000201,0.000061,0.000181,0.000034,0.000047,0.001472
Flexistipes.sinusarabici,0.000000,0.000106,0.000034,0.000000,0.000055,0.000127,0.000117,0.000095,0.000000,0.000022,...,0.000000,0.000000,0.000083,0.000000,0.000000,0.000030,0.000060,0.000000,0.000019,0.000060
Lactococcus.phage.50101,0.000094,0.000029,0.000078,0.000000,0.000109,0.000042,0.000221,0.000000,0.000032,0.000112,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000060,0.000000,0.000009,0.000000
Salmonella.phage.vB_SosS_Oslo,0.000031,0.000019,0.000268,0.000023,0.000055,0.000000,0.000078,0.000000,0.000032,0.000000,...,0.000000,0.000000,0.000000,0.000041,0.000000,0.000030,0.000000,0.000000,0.000009,0.000000
Cellulophaga.phage.phi38.1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000021,0.000041,0.000000,0.000000,0.000020,0.000000,0.000000,0.000060
Escherichia.phage.vB_EcoM_Alf5,0.000000,0.000000,0.000000,0.000023,0.000000,0.000042,0.000000,0.000024,0.000032,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000030,0.000000,0.000000,0.000000,0.000000
Geobacillus.sp..JS12,0.020768,0.001666,0.001778,0.004491,0.008493,0.045619,0.002275,0.007470,0.031929,0.005252,...,0.004983,0.002771,0.008089,0.005734,0.006158,0.006968,0.005817,0.006504,0.002344,0.007751
Mycobacterium.phage.Charlie,0.000000,0.000000,0.000000,0.000000,0.000027,0.000000,0.000000,0.000000,0.000032,0.000000,...,0.000058,0.000053,0.000041,0.000000,0.000029,0.000000,0.000000,0.000000,0.000000,0.000060
Yellowstone.lake.mimivirus,0.000031,0.000029,0.000022,0.000304,0.000027,0.000169,0.000052,0.000143,0.000065,0.000022,...,0.000000,0.000026,0.000103,0.000041,0.000057,0.000061,0.000020,0.003594,0.000000,0.000090


In [13]:
# change the species name by NCBI name
# Change the row indexes
dataG.index = ['Arthrobacter phage KellEzio', 'Escherichia phage PBECO4', 'Flexistipes sinusarabici', 'Lactococcus phage 50101',
             'Salmonella phage vB_SosS_Oslo','Cellulophaga phage phi38:1', 'Escherichia phage vB_EcoM_Alf5','Geobacillus sp. JS12',
              'Mycobacterium phage Charlie', 'Yellowstone lake mimivirus']
dataG # matrix
dat # metadata

# this will we the input matrix and metadata from GAIA

,Sample names,condition1
0,SRR15853012,1
1,SRR15852941,1
2,SRR15852948,1
3,SRR15853009,1
4,SRR15853002,1
5,SRR15852986,1
6,SRR15852987,1
7,SRR15852998,1
8,SRR15853011,1
9,SRR15853000,1


In [14]:
# This 2 files will we our input in our python script ML_product1
# save as a cvs file
pd.DataFrame(dataG).to_csv("matrix.csv") # input matrix --> will be saved as matrixHuge
pd.DataFrame(dat).to_csv("metadata.csv") # input metadata --> will be saved as metadataHuge



In [28]:
# input matrix and metadata to our Python script
#dataG
dat


,Sample names,condition1
0,SRR15853012,1
1,SRR15852941,1
2,SRR15852948,1
3,SRR15853009,1
4,SRR15853002,1
5,SRR15852986,1
6,SRR15852987,1
7,SRR15852998,1
8,SRR15853011,1
9,SRR15853000,1


In [ ]:
##### Jupiter notebook code to put in our python script #####

In [15]:

# *this part will go in to the python script ML_product

# loading the data in python script

matrix = pd.read_csv('matrix.csv')
metadata = pd.read_csv('metadata.csv')

matrix


,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,30,31,32,33,34,35,36,37,38,39
0,Arthrobacter phage KellEzio,0.000156,0.000048,0.000011,0.000023,0.000055,0.000127,0.000078,0.000072,0.000129,...,0.000000,0.000000,0.000144,0.000020,0.000029,0.000000,0.000020,0.000034,0.000028,0.000210
1,Escherichia phage PBECO4,0.000407,0.000058,0.000056,0.000070,0.000355,0.000000,0.000065,0.000430,0.000485,...,0.000232,0.000079,0.000083,0.000061,0.000201,0.000061,0.000181,0.000034,0.000047,0.001472
2,Flexistipes sinusarabici,0.000000,0.000106,0.000034,0.000000,0.000055,0.000127,0.000117,0.000095,0.000000,...,0.000000,0.000000,0.000083,0.000000,0.000000,0.000030,0.000060,0.000000,0.000019,0.000060
3,Lactococcus phage 50101,0.000094,0.000029,0.000078,0.000000,0.000109,0.000042,0.000221,0.000000,0.000032,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000060,0.000000,0.000009,0.000000
4,Salmonella phage vB_SosS_Oslo,0.000031,0.000019,0.000268,0.000023,0.000055,0.000000,0.000078,0.000000,0.000032,...,0.000000,0.000000,0.000000,0.000041,0.000000,0.000030,0.000000,0.000000,0.000009,0.000000
5,Cellulophaga phage phi38:1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000021,0.000041,0.000000,0.000000,0.000020,0.000000,0.000000,0.000060
6,Escherichia phage vB_EcoM_Alf5,0.000000,0.000000,0.000000,0.000023,0.000000,0.000042,0.000000,0.000024,0.000032,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000030,0.000000,0.000000,0.000000,0.000000
7,Geobacillus sp. JS12,0.020768,0.001666,0.001778,0.004491,0.008493,0.045619,0.002275,0.007470,0.031929,...,0.004983,0.002771,0.008089,0.005734,0.006158,0.006968,0.005817,0.006504,0.002344,0.007751
8,Mycobacterium phage Charlie,0.000000,0.000000,0.000000,0.000000,0.000027,0.000000,0.000000,0.000000,0.000032,...,0.000058,0.000053,0.000041,0.000000,0.000029,0.000000,0.000000,0.000000,0.000000,0.000060
9,Yellowstone lake mimivirus,0.000031,0.000029,0.000022,0.000304,0.000027,0.000169,0.000052,0.000143,0.000065,...,0.000000,0.000026,0.000103,0.000041,0.000057,0.000061,0.000020,0.003594,0.000000,0.000090


In [16]:

matrixT = matrix.transpose()

#matrix.to_csv('matrix.csv', header=False, index=False)
matrixT


,0,1,2,3,4,5,6,7,8,9
Unnamed: 0,Arthrobacter phage KellEzio,Escherichia phage PBECO4,Flexistipes sinusarabici,Lactococcus phage 50101,Salmonella phage vB_SosS_Oslo,Cellulophaga phage phi38:1,Escherichia phage vB_EcoM_Alf5,Geobacillus sp. JS12,Mycobacterium phage Charlie,Yellowstone lake mimivirus
0,0.000156,0.000407,0.0,0.000094,0.000031,0.0,0.0,0.020768,0.0,0.000031
1,0.000048,0.000058,0.000106,0.000029,0.000019,0.0,0.0,0.001666,0.0,0.000029
2,0.000011,0.000056,0.000034,0.000078,0.000268,0.0,0.0,0.001778,0.0,0.000022
3,0.000023,0.00007,0.0,0.0,0.000023,0.0,0.000023,0.004491,0.0,0.000304
4,0.000055,0.000355,0.000055,0.000109,0.000055,0.0,0.0,0.008493,0.000027,0.000027
5,0.000127,0.0,0.000127,0.000042,0.0,0.0,0.000042,0.045619,0.0,0.000169
6,0.000078,0.000065,0.000117,0.000221,0.000078,0.0,0.0,0.002275,0.0,0.000052
7,0.000072,0.00043,0.000095,0.0,0.0,0.0,0.000024,0.00747,0.0,0.000143
8,0.000129,0.000485,0.0,0.000032,0.000032,0.0,0.000032,0.031929,0.000032,0.000065


In [17]:
# first row be the columns names

matrixT.rename(columns=matrixT.iloc[0], inplace = True)


In [18]:
matrixFinal = matrixT.iloc[1: , :]

In [19]:
matrixFinal

,Arthrobacter phage KellEzio,Escherichia phage PBECO4,Flexistipes sinusarabici,Lactococcus phage 50101,Salmonella phage vB_SosS_Oslo,Cellulophaga phage phi38:1,Escherichia phage vB_EcoM_Alf5,Geobacillus sp. JS12,Mycobacterium phage Charlie,Yellowstone lake mimivirus
0,0.000156,0.000407,0.0,0.000094,0.000031,0.0,0.0,0.020768,0.0,0.000031
1,0.000048,0.000058,0.000106,0.000029,0.000019,0.0,0.0,0.001666,0.0,0.000029
2,0.000011,0.000056,0.000034,0.000078,0.000268,0.0,0.0,0.001778,0.0,0.000022
3,0.000023,0.00007,0.0,0.0,0.000023,0.0,0.000023,0.004491,0.0,0.000304
4,0.000055,0.000355,0.000055,0.000109,0.000055,0.0,0.0,0.008493,0.000027,0.000027
5,0.000127,0.0,0.000127,0.000042,0.0,0.0,0.000042,0.045619,0.0,0.000169
6,0.000078,0.000065,0.000117,0.000221,0.000078,0.0,0.0,0.002275,0.0,0.000052
7,0.000072,0.00043,0.000095,0.0,0.0,0.0,0.000024,0.00747,0.0,0.000143
8,0.000129,0.000485,0.0,0.000032,0.000032,0.0,0.000032,0.031929,0.000032,0.000065
9,0.000179,0.000313,0.000022,0.000112,0.0,0.0,0.0,0.005252,0.0,0.000022


In [20]:
# in metadata select only the binary column

metadata = metadata.loc[:, metadata.columns.str.startswith('condition1')]
metadata


,condition1
0,1
1,1
2,1
3,1
4,1
5,1
6,1
7,1
8,1
9,1


In [21]:
# train Random Forest with LOOCV

# Independent variable
X = matrixFinal # removed last column (condition)

# Target variable
y = metadata # explicity selecting the last column

cv = LeaveOneOut()
model = RandomForestClassifier()

# use the model to predict the response value of the one observation
# left out of the model and calculate the mean squared error (MSE)
scores = cross_val_score(model,  X , y , cv = cv, scoring='neg_mean_squared_error',n_jobs = -1) 
scores_acc = cross_val_score(model,  X , y , cv = cv, scoring='accuracy',n_jobs = -1) 


#view RMSE
print('mean squared error: ' + str(sqrt(mean(absolute(scores)))))
# accuracy
print('Accuracy: %.3f (%.3f)' % (mean(scores_acc), std(scores_acc)))




mean squared error: 0.3872983346207417
Accuracy: 0.925 (0.263)


In [22]:
# Save the model

# To fit our stimator (call fit in it explicity with provided dataset)
model.fit(X, y.values.ravel())
# To serialize
import pickle


with open('modelRF.pkl', 'wb') as fid:
    pickle.dump(model, fid, protocol = 2)
    
# To deserialize estimator later
with open('modelRF.pkl', 'rb') as fid:
    model = pickle.load(fid)